In [1]:
require(caTools)
library(data.table)
library(nlme)
library(caret)
library(ggplot2)
library(dplyr)

Loading required package: caTools

Loading required package: ggplot2

Loading required package: lattice

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘dplyr’


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
#1.MAT...................
#Import data from disk dataset after RFE & VIF
sentinel_site_ADF <- read.csv('/home/eoafrica/FATIMA_Project/M2021/BD_BH_VIF/bd_bh_adf.csv', header = TRUE)
sentinel_site_ADF <- sentinel_site_ADF[2:7]
names(sentinel_site_ADF)

[1] "GRCI"    "VIgreen" "NDSVI"   "RVI3"    "RVI2"    "ADF"

In [3]:
model_list_ADF<- list()
resultList_ADF <- data.frame()
#Train lm with all combinations
list_comb_ADF=list() # je crÃ©e une liste pour stocker les combinaisons
idx=1 # la liste commence pas l'indice 1
for (x in 2:5){
  new_comb_ADF = combn(c("GRCI", 'VIgreen', "NDSVI", 'RVI3', "RVI2"), x) # as.data.frame: pas la peine de mettre cela sous forme d'un data frame juste la combn fait sortir cela en array
  for (i in 1: dim (new_comb_ADF)[2]) {
    list_comb_ADF[idx]= paste(new_comb_ADF[,i], collapse="+")# la combinaison a mettre a chaque ligne 
    idx = idx + 1
    
  }
} 
#print(list_comb_ADF)

In [8]:
MLM_formula = paste("ADF", list_comb_ADF , sep="~")

for (j in 1:length(MLM_formula)){
  set.seed(102)
     reg_model_ADF <- lm(MLM_formula[15], data= sentinel_site_ADF)
     result_ADF <- summary(reg_model_ADF)
     resultList_ADF[j,1] <- (MLM_formula)[j]
     resultList_ADF[j,2] <- result_ADF$r.squared
    
}  

names(resultList_ADF) <- c("MLM","R2cal")

In [9]:
#Validation croisee des modeles de regression lineaire
model_list_ADF<- list()
resultList_ADF_train <- data.frame()
predictions_list <- list()  # Pour stocker les prédictions de chaque fold

for (m in 1:length(MLM_formula)){
  set.seed(102)
  trControl = trainControl(method = 'repeatedcv', number = 5, repeats = 3, verboseIter = FALSE, savePredictions=TRUE) #Preciser le schema de reechantillonnage
  Fit_model_ADF <- train(formula(MLM_formula[m]), data=sentinel_site_ADF, method ="lm", trControl = trControl) #Ajustement de modeles
  model_list_ADF = as.list.data.frame(Fit_model_ADF)
  model_list_ADF <- c(model_list_ADF, list(Fit_model_ADF))
  # Calcul des métriques de validation
  result_ADF_train <- summary(model_list_ADF)
  resultList_ADF_train[m,1] <- (MLM_formula)[m]
  resultList_ADF_train[m,2] <- model_list_ADF$results[1,2]
  resultList_ADF_train[m,3] <- model_list_ADF$results[1,3]
  resultList_ADF_train[m,4] <- model_list_ADF$results[1,4]
  actual_values <- sentinel_site_ADF$ADF 
  rmse <- resultList_ADF_train[m,4]
  mean_actual_values <- mean(actual_values)
  resultList_ADF_train[m,5] <- (rmse/mean_actual_values)*100
}

In [10]:

names(resultList_ADF_train) <- c("MLM_formula", "RMSE", "R2_Val", "MAE", "RRMSE")

resultList_ADF_final<-cbind(resultList_ADF, resultList_ADF_train[3],resultList_ADF_train[2],resultList_ADF_train[4],resultList_ADF_train[5])  
resultList_ADF_final<-resultList_ADF_final[order(resultList_ADF_final$RMSE), c(1:6)] # l'objectif ici c'est que R nous trie notre colonne de RMSE du plus petit au plus grand

In [11]:
print(resultList_ADF_final)

                                MLM     R2cal    R2_Val     RMSE      MAE
15              ADF~GRCI+NDSVI+RVI2 0.6205591 0.6547017 2.404050 1.962277
2                    ADF~GRCI+NDSVI 0.6205591 0.6214024 2.480111 2.150863
24         ADF~GRCI+NDSVI+RVI3+RVI2 0.6205591 0.6192497 2.614243 2.130063
11           ADF~GRCI+VIgreen+NDSVI 0.6205591 0.6208149 2.619303 2.198649
22      ADF~GRCI+VIgreen+NDSVI+RVI2 0.6205591 0.6292421 2.639905 2.175226
14              ADF~GRCI+NDSVI+RVI3 0.6205591 0.5915784 2.648054 2.306500
21      ADF~GRCI+VIgreen+NDSVI+RVI3 0.6205591 0.5809598 2.800534 2.311133
26 ADF~GRCI+VIgreen+NDSVI+RVI3+RVI2 0.6205591 0.5986654 2.893100 2.380757
8                    ADF~NDSVI+RVI3 0.6205591 0.4320580 2.916556 2.557472
4                     ADF~GRCI+RVI2 0.6205591 0.4368185 2.969522 2.544743
3                     ADF~GRCI+RVI3 0.6205591 0.4474447 2.993842 2.644105
17           ADF~VIgreen+NDSVI+RVI3 0.6205591 0.4348579 3.018632 2.532889
20              ADF~NDSVI+RVI3+RVI2 0.

In [ ]:
# Exportation des resultats sous format csv
write.csv(resultList_ADF_final, "/home/eoafrica/FATIMA_Project/M2021/BD_BH_VIF/Result_MLM_inter_effect/result_MLM_inter_effect_ADF.csv")

In [ ]:
Fit_model_ADF_15 <- train(formula(MLM_formula[15]), data=sentinel_site_ADF, method ="lm", trControl = trControl)

In [ ]:
df<-Fit_model_ADF_15$pred
names(df)

In [ ]:
# Calcul de la droite de régression linéaire
lm_model <- lm(df$pred~df$obs)
summary(lm_model)

In [ ]:
folds_adf <- df[1:2]
colnames(folds_adf) <- c("ADF_estime", "ADF_observe")
names(folds_adf)

write.csv(results, "/home/eoafrica/FATIMA_Project/M2021/BD_BH_VIF/Result_MLM_HQ_pred/result_MLM_pred_ADF.csv")

In [ ]:
library(ggplot2)

In [ ]:
# Créer le scatter plot avec ggplot2
plt2 <- ggplot(folds_adf, aes(x = ADF_estime, y = ADF_observe)) +
    geom_point(size = 3) +
    labs(x = expression(bold("ADF herbacés observées (%MS)")), 
         y = expression(bold("ADF herbacés estimées (%MS)"))) +
    theme(axis.text=element_text(size=12, color="black", face='bold'),
    axis.title=element_text(size=15, color="black", face="bold"),
    plot.title = element_text(size = 20, face = "bold")) +
    scale_shape_manual(values = c("ADF" = 16), name = "Variables") +
    geom_smooth(aes(color = "ADF"), method = "lm", se = TRUE) +
    scale_color_manual(values = c("orange"), 
                       labels = c("ADF"), 
                       name = "Variables") +
    geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "black") +
    guides(shape = guide_legend(title = "Variables")) +
    geom_text(aes(x = 19, y = 30, label = "p-value < 0,05"), color = "black", size = 5.5) +
    geom_text(aes(x = 19, y = 30.5, label = "R² = 0.5547"), color = "black", size = 5.5)
plt2

In [ ]:
ggsave(plt2, filename = "/home/eoafrica/FATIMA_Project/M2021/BD_BH_VIF/Result_MLM_HQ_pred/graph_reg_obs_folds_ADF_HQ.png", device ='png')